# collaboration with Woosok Moon

In [ ]:
import os
import sys
import numpy as np
import xesmf as xe
import xarray as xr
import pandas as pd
import cmocean
import cartopy
import cartopy.crs as ccrs
import matplotlib as mpl
import matplotlib
import matplotlib.pyplot as plt

In [ ]:
%matplotlib inline
%config InlineBackend.print_figure_kwargs={'bbox_inches':None}
matplotlib.rc_file('../rc_file')
%load_ext autoreload
%autoreload 2
%aimport - numpy - scipy - matplotlib.pyplot

In [ ]:
sys.path.append("..")
from tqdm import tqdm_notebook
from maps import rect_polygon
from paths import path_results, path_prace, path_data
from filters import lowpass
from regions import SST_index_bounds, mask_box_in_region
from xr_DataArrays import xr_AREA
from ab_derivation_SST import DeriveSST as DS

# PDO vs SO index time series in observations, high, and low resolution CESM

In [ ]:
ax = plt.axes(projection=ccrs.PlateCarree(central_longitude=275))
ax.stock_img()
for i, bounds in enumerate([SST_index_bounds('PDO'), (-30,80,-80,-50)]):
    ax.add_patch(mpl.patches.Polygon(xy=rect_polygon(bounds),
                                  facecolor='none', edgecolor=f'C{i}',
                                  linewidth=2, zorder=2,
                                  transform=ccrs.PlateCarree(), ), )
plt.title('PDO and SOi regions')
plt.tight_layout()

### PDO time series

In [ ]:
dateparse = lambda x: pd.datetime.strptime(x, '%Y%m')
PDO = pd.read_csv(f'{path_data}/PDO/PDO_NCDC_NOAA.csv', skiprows=1, index_col=0, date_parser=dateparse).to_xarray().to_array().rename({'Date':'time'}).squeeze()
PDO['time'] = PDO.time.dt.year.astype(np.float64)+(PDO.time.dt.month/12-1/24).astype(np.float64)

In [ ]:
PC1_had  = -xr.open_dataset(f'{path_prace}/SST/PMV_EOF_20N_had.nc', decode_times=False).pcs.sel(mode=0)
PC1_ctrl = xr.open_dataset(f'{path_prace}/SST/PMV_EOF_20N_ctrl_51_301.nc', decode_times=False).pcs
PC1_lpd  = xr.open_dataset(f'{path_prace}/SST/PMV_EOF_20N_lpd_154_404.nc', decode_times=False).pcs.sel(mode=0)

In [ ]:
plt.plot(PDO.time, PDO)
plt.plot(PC1_had.time/365+1870, PC1_had)

### SO index time series
30W-80E, 50-80S

In [ ]:
fn = f'{path_results}/Woosok/PDO_SOi_timeseries.nc'
if os.path.exists(fn):
    ds = xr.open_dataset(fn)
else:
    SST_ctrl = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ds_dt_ctrl_51_301.nc', decode_times=False)
    SST_ctrl = DS().shift_ocn_rect(SST_ctrl)
    SST_lpd  = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ds_dt_lpd_154_404.nc', decode_times=False)
    SST_had  = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ds_dt_had.nc'        , decode_times=False)

    AREA_rect = xr_AREA(domain='ocn_rect')
    AREA_low  = xr_AREA(domain='ocn_low')
    AREA_had  = xr_AREA(domain='ocn_had')

    SOi_MASK_had  = mask_box_in_region(domain='ocn_had' , mask_nr=1, bounding_lats=(-80,-50), bounding_lons=(0,80))
    SOi_MASK_rect = mask_box_in_region(domain='ocn_rect', mask_nr=1, bounding_lats=(-80,-50), bounding_lons=(0,80))
    SOi_MASK_low  = mask_box_in_region(domain='ocn_low' , mask_nr=1, bounding_lats=(-80,-50), bounding_lons=(0,80))

    # had
    AREA_had_SO = AREA_had.where(SOi_MASK_had).sum(dim=['latitude','longitude'])
    print(AREA_had_SO.values)
    SOi_had = (AREA_had*SST_had).where(SOi_MASK_had).sum(dim=['latitude','longitude'])/AREA_had_SO
    SOi_had /= SOi_had.std(dim='time')

    # lpd
    AREA_low_SO = AREA_low.where(SOi_MASK_low).sum(dim=['nlat','nlon'])
    print(AREA_low_SO.values)
    SOi_lpd = (AREA_low*SST_lpd).where(SOi_MASK_low).sum(dim=['nlat','nlon'])/AREA_low_SO
    SOi_lpd /= SOi_lpd.std(dim='time')

    # ctrl
    AREA_rect_SO = AREA_rect.where(SOi_MASK_rect).sum(dim=['t_lat','t_lon'])
    print(AREA_rect_SO.values)
    SOi_ctrl = (AREA_rect*SST_ctrl).where(SOi_MASK_rect).sum(dim=['t_lat','t_lon'])/AREA_rect_SO
    SOi_ctrl /= SOi_ctrl.std(dim='time')

    ds = xr.Dataset()
    ds['CESM_high_PDO'] = PC1_ctrl.squeeze().assign_coords(time=np.arange(3000))
    ds['CESM_high_SOi'] = SOi_ctrl.squeeze().assign_coords(time=np.arange(3000))
    ds['CESM_low_PDO']  = PC1_lpd .squeeze().assign_coords(time=np.arange(3000))
    ds['CESM_low_SOi']  = SOi_lpd .squeeze().assign_coords(time=np.arange(3000))
    ds['HadISST_PDO']   = PC1_had .assign_coords(time=np.arange(1788))
    ds['HadISST_SOi']   = SOi_had .assign_coords(time=np.arange(1788))
    ds.to_netcdf(fn)

In [ ]:
f, ax = plt.subplots(3,1, figsize=(10,7))
for i, run in enumerate(['had', 'ctrl', 'lpd']):
    key = ['HadISST', 'CESM_high', 'CESM_low'][i]
    offset = [1870,51,154][i]
    tss = [ds[f'{key}_PDO'].dropna(dim='time'), ds[f'{key}_SOi'].dropna(dim='time')]
    print(len(tss[0]), len(tss[1]))
    cor_f = np.corrcoef(lowpass(tss[0], 13*12)[7*12:-7*12].values.flatten(), lowpass(tss[1], 13*12)[7*12:-7*12].values.flatten())[0,1]
    cor   = np.corrcoef(tss[0].values.flatten(), tss[1].values.flatten())[0,1]
    for j, ts in enumerate(tss):
        ax[i].plot(ts.time/12+offset, ts, lw=.5, alpha=.5, c=f'C{j}')
        ax[i].plot(ts.time[7*12:-7*12]/12+offset, lowpass(ts, 13*12)[7*12:-7*12], c=f'C{j}', label=['PDO', 'SOi'][j])
    ax[i].plot([0,1],[0,1], c='darkgrey', lw=.5, alpha=.5, label=f'unfiltered corr={cor:.2f}')
    ax[i].plot([0,1],[0,1], c='darkgrey', label=f'13 year lowpass corr={cor_f:.2f}')
    ax[i].legend(loc=2, ncol=2)
    ax[i].set_xlim([(1845,2105),(45,305),(150,410)][i])
    ax[i].set_ylim((-4,4))
    
ax[0].set_ylabel('two-factor detrended HadISST')
ax[0].set_xlabel('time [years C.E.]')
ax[1].set_ylabel('high res. CESM')
ax[2].set_ylabel('low res. CESM')
for i in np.arange(1,3):  ax[i].set_xlabel('time [model years]')
plt.tight_layout()
plt.savefig(f'{path_results}/Woosok/PDO_SOi_overview')

### SST anomalies in the three defined by Woosok

![title](../../results/Woosok/Woosok_SST_lat_lon.png)

In [ ]:
SST_had_orig = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_had.nc')
SST_had = xr.open_dataarray(f'{path_prace}/SST/SST_monthly_ds_dt_had.nc')

In [ ]:
f = plt.figure(figsize=(12,4.2))
lats, lons = SST_had.latitude, SST_had.longitude
lons, lats = np.meshgrid(lons, lats)
for i, period in enumerate([(1958,1976),(1977,2000),(2001,2018)]):
    SST_anomaly = SST_had.sel(time=slice(f'{period[0]}-01-01',f'{period[1]}-12-31')).mean(dim='time')
    ax = f.add_subplot(1, 4, i+1, projection=ccrs.Orthographic(central_latitude=-90, central_longitude=180))
    ax.text(.5, 1.05, f'{period[0]}-{period[1]}', transform=ax.transAxes, fontsize=16, ha='center')
    ax.set_position([.01+i*.315,.01,.31,.9])
    im = ax.pcolormesh(lons, lats, SST_anomaly, cmap='cmo.balance',
                       vmin=-.4, vmax=.4, transform=ccrs.PlateCarree())
    ax.coastlines(resolution='110m')
    ax.gridlines()
    ax.add_patch(mpl.patches.Polygon(xy=rect_polygon((-30,80,-80,-50)),
                                  facecolor='none', edgecolor=f'C1',
                                  linewidth=2, zorder=2,
                                  transform=ccrs.PlateCarree(), ), )
    
ax = f.add_subplot(1, 4, 4)
ax.set_position([.955,.1,.01,.75])
cbar = plt.colorbar(im, cax=ax, shrink=.9, pad=.0, orientation='vertical', extend='both')
# cbar.ax.set_yticklabels(np.arange(-.4,.5,.1), fontsize=12)

In [ ]:
latS, latN, lonW, lonE = -80, -50, -30, 80
SOI_orig = SST_had_orig.sel(longitude=slice(lonW,lonE)).sel(latitude=slice(latN,latS)).mean(dim=('latitude', 'longitude'))
SOI_orig_anomaly = SOI_orig - SOI_orig.sel(time=slice(pd.datetime(year=1958, month=1, day=1),pd.datetime(year=2018, month=12, day=31))).mean(dim='time')

plt.title('effect of two-factor detrending on SOi')
plt.axhline(0, lw=.5, c='k')
SST_had.sel(longitude=slice(lonW,lonE)).sel(latitude=slice(latN,latS)).mean(dim=('latitude', 'longitude')).plot(alpha=.3)
lowpass(SST_had.sel(longitude=slice(lonW,lonE)).sel(latitude=slice(latN,latS)).mean(dim=('latitude', 'longitude')),13).plot(c='C0')
lowpass(SOI_orig_anomaly,13).plot()
plt.gca().axvspan(pd.datetime(year=1958, month=1, day=1),pd.datetime(year=1976, month=12, day=31), alpha=0.5, color='g')
plt.gca().axvspan(pd.datetime(year=1977, month=1, day=1),pd.datetime(year=2000, month=12, day=31), alpha=0.5, color='red')
plt.gca().axvspan(pd.datetime(year=2001, month=1, day=1),pd.datetime(year=2018, month=12, day=31), alpha=0.5, color='g')

## identifying time periods

![title](../../results/Woosok/Woosok_phases.png)

SOi (solid), PDO (dashed)

In [ ]:
PDO = pd.read_csv(f'{path_data}/PDO/PDO_NCDC_NOAA.csv', skiprows=1, index_col=0, date_parser=dateparse).to_xarray().to_array().rename({'Date':'time'}).squeeze()
# PDO['time'] = PDO.time.dt.year.astype(np.float64)+(PDO.time.dt.month/12-1/24).astype(np.float64)

In [ ]:
PDO.time

In [ ]:
f, ax = plt.subplots(1,1, figsize=(8,2), constrained_layout=True)
ax.axhline(0, lw=.5, c='k')
ax.axvline(1958)
ax.axvline(1980)
ax.axvline(2009)
key = 'HadISST'
offset = [1870,51,154][i]
tss = [-ds[f'{key}_PDO'].dropna(dim='time'), ds[f'{key}_SOi'].dropna(dim='time')]
for j, ts in enumerate(tss):
    ax.plot(ts.time[7*12:-7*12]/12+offset, lowpass(ts, 10*12)[7*12:-7*12], c=f'C{j}', label=['PDO detrended', 'SOi detrended'][j], ls=['--','-.'][j])
ax.set_xlim([(1957.5,2018)][i])
ax.plot(PDO.time.dt.year+PDO.time.dt.month/12, lowpass(PDO,10*12), ls=':', c='C0', label='PDO original')
ax.plot(SST_had_orig.time.dt.year+SST_had_orig.time.dt.month/12, lowpass(SOI_orig_anomaly,10*12)*10, ls='-', c='C1', label=r'SOi original ($\times$10)')
ax.legend(ncol=2)
ax.set_ylabel('SO & PDO')
ax.set_xlabel('time [years C.E.]')
plt.ylim((-2,2))

- it is not clear how the data was detrended in Woosok's case, this is very important for the SOi (cf first crossover point)
- PDO: similat between detrended & original & Woosok's

### lead-lag  analysis between signals

### identifying times in the model

### OHC anomalies